</div>
<div align="left">
  <img src="img/abstract.png" width="400" alt="Funny little diagram">
  <p><em> Evolve nodes, evolve plans, and learn from the best performing ones.</em></p>
</div>
<div align="center">
</em></p>
</div>

#### Node Initialization (Refactoring ...)

In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-rimbwy29f75muq"
api_key = os.environ["RUNPOD_API_KEY"]
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

Could not load vllm class, check CUDA support and GPU RAM size


In [2]:
get_endpoint_response(["hi"])

Processing LLM queries: 100%|██████████| 1/1 [00:22<00:00, 22.57s/it]

 :: Total time elapsed: 22.58s, 0 errors


['It\'s a pleasure to meet you. I\'m honored to have received the Turing Award, which is often considered the "Nobel Prize of Computing." It\'s a recognition of the impact and influence that our work in computer science has had on the world.\n\nI\'d be happy to chat with you about computer science, technology, or any other topic that interests you. What would you like to talk about?']

In [ ]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.evolnode import EvolNode
from methods.llm import get_groq_response, get_claude_response

# Code + Compilor Task
# mp = MetaPrompt("Search for age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.CODE)
# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

test_inputs = [c[0] for c in test_cases]

node = EvolNode(mp, None, None, get_response=get_endpoint_response, test_cases=test_cases) # setting manual test cases

node.evolve("i1", replace=True, batch_size=20, num_runs=2, print_summary=True) # Scale up batch size


input_dict = {"name": "Dilireba"}
node.get_response = get_groq_response # fast sequential inference 
output_dict = node(input_dict) # use node as a function
print("Output dict: ", output_dict)

</div>
<div align="center">
  <img src="img/Project-Nirvana-evolve.gif" width="500" alt="Fourier reconstruction convergence">
  <p><em> Evolve a population of nodes. </em></p>
</div>

In [ ]:
# Population building phase ... 
from methods.llm import get_groq_response, get_claude_response
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.population import Evolution

mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

evo = Evolution(pop_size=20, meta_prompt=mp, get_response=get_endpoint_response, 
                test_cases=test_cases, max_attempts=3, num_eval_runs=2,
                load=True)

strategies = ["m2"] # ["i1", "i1", "m2", "e2"]
evo.get_offspring(strategies)

evo.chat("How effective is the current evolution strategy? What improvement has it made in terms of fitness, and in terms of the implementation?",
         get_claude_response) 

# code-based check 
print(evo.population_info)

Processing queries: 100%|██████████| 20/20 [00:16<00:00,  1.20it/s]


 :: Total time elapsed: 16.71s, 0 errors


Processing queries: 100%|██████████| 34/34 [00:28<00:00,  1.19it/s]


 :: Total time elapsed: 28.46s, 0 errors


Processing queries: 0it [00:00, ?it/s]


 :: Total time elapsed: 0.00s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.33
  🎯 Functional fitness: 1.00
  ⭐ Global fitness:     0.67
  🔄 Batch size:        17
  ⏱️ Time taken: 85.64 seconds


📊 Code 0: Fitness: 16.7%
--------------------------------------------------------------------------------
❌ Error Messages:
Input: {'name': 'Dilireba'}, prediction is not aligned with expected output, Expected: {'age': 32} Predicted: {'age': 87}, Error message: 
Value mismatch for key age: 87 != 32


Input: {'name': 'ChengXiao'}, prediction is not aligned with expected output, Expected: {'age': 26} Predicted: {'age': 33}, Error message: 
Value mismatch for key age: 33 != 26




📊 Code 1: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Failed to parse LLM response -- No JSON structure found in the provided text.
Failed to parse LLM response -- JsonDecodeError : 
Expecting value: line 1 column 1 (char 0)AstL


</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="900" alt="Fourier reconstruction convergence">
  <p><em> Plan, and evolve the plans. </em></p>
</div>


In [ ]:
from methods.llm import get_claude_response, get_groq_response
from methods.diagram import visualize_plan_dict
from methods.meta_prompt import MetaPlan, PromptMode, MetaPrompt
from methods.evolnode import PlanNode, EvolNode
mp = MetaPrompt("Collect AI papers from arXiv in the given time frame and save it in output directory", "get_arxiv_papers", ["start_date", "end_date", "output_dir"], ["num_papers", "time_taken"], ["str", "str", "str"], ["int", "int"], PromptMode.CODE)
main_test_cases = [({"start_date": "2024-11-17", "end_date": "2024-11-18", "output_dir": "tmp"}, {"num_papers": 194, "time_taken": 480})]
def useless_metric(x, y):
    return x, ""
node = EvolNode(mp, get_response=get_endpoint_response, test_cases=main_test_cases, custom_metric_map={"time_taken": useless_metric, "num_papers": useless_metric}
)
node.evolve("i1", replace=True, batch_size=100, num_runs=1, print_summary=True, timeout=True, feedback="To get the id, do result.get_short_id()")

2024-11-29 20:23:27.466926: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


     :: Query time: 18.37s


Processing LLM queries: 100%|█████████████████| 100/100 [01:23<00:00,  1.19it/s]


 :: Total time elapsed: 83.77s, 0 errors
ERROR PARSING CODE
ERROR PARSING CODE
     :: Evolution time: 84.80s


  0%|                                                   | 0/999 [00:00<?, ?it/s]












































100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 36.8kiB/s]
100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 36.0kiB/s]
100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 33.8kiB/s]
100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 36.5kiB/s]
100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 36.6kiB/s]
100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 37.9kiB/s]
100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 41.0kiB/s]
100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 37.7kiB/s]
100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 32.1kiB/s]
100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 29.3kiB/s]




























100%|████████████████████████████████

...

Error downloading file: name 'os' is not defined
Error downloading file: name 'os' is not defined
Error downloading file: name 'os' is not defined











100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 40.7kiB/s]
100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 42.1kiB/s]
100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 49.2kiB/s]
100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 65.6kiB/s]
100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 66.0kiB/s]
100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 70.0kiB/s]


Error downloading file: name 'os' is not defined
Error downloading file: name 'os' is not defined
Error downloading file: name 'os' is not defined
Error downloading file: name 'os' is not defined
Error downloading file: name 'os' is not defined
Error downloading file: name 'os' is not defined


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/shah.mahir/opt/anaconda3/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shah.mahir/opt/anaconda3/lib/python3.12/multiprocessing/spawn.py", line 130, in _main
    preparation_data = reduction.pickle.load(from_parent)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
EOFError: Ran out of input









100%|██████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 117kiB/s]
100%|██████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 145kiB/s]
100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 94.5kiB/s]






100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 54.8kiB/s]
100%|█████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 46.7kiB/s]
100%|██████████████████████████████████████| 1.85k/1.85k [00:00<00:00, 1

Paper 2411.10983v1 saved successfully.
Paper 2411.11976v1 saved successfully.
Paper 2411.13585v1 saved successfully.
Paper 2411.10995v1 saved successfully.
Paper 2411.14463v1 saved successfully.
Paper 2411.11449v1 saved successfully.
Paper 2411.11478v2 saved successfully.
Paper 2411.11774v1 saved successfully.
Paper 2411.11795v1 saved successfully.
Paper 2411.11235v1 saved successfully.
Paper 2411.11494v1 saved successfully.
Paper 2411.12128v3 saved successfully.
Paper 2411.14472v1 saved successfully.
Paper 2411.11752v1 saved successfully.
Paper 2411.11835v1 saved successfully.
Paper 2411.11221v1 saved successfully.
Paper 2411.11613v2 saved successfully.
Paper 2411.17712v1 saved successfully.
Paper 2411.11451v1 saved successfully.
Paper 2411.11786v1 saved successfully.
Paper 2411.12090v1 saved successfully.
Paper 2411.11783v1 saved successfully.
Paper 2411.11910v2 saved successfully.
Paper 2411.11940v2 saved successfully.
Paper 2411.10939v1 saved successfully.
Paper 2411.11045v1 saved 

In [ ]:
from methods.llm import get_claude_response, get_groq_response
from methods.diagram import visualize_plan_dict
from methods.meta_prompt import MetaPlan
from methods.evolnode import PlanNode


# Initialize PlanNode 
mp = MetaPlan("Collect AI papers from arXiv in the given time frame and save it in output directory", "get_arxiv_papers", ["start_date", "end_date", "output_dir"], ["num_papers", "time_taken"], ["str", "str", "str"], ["int", "int"])
plan = EvolNode(mp, get_endpoint_response)

# i1 evolution of plan
plan_dicts, err_msg = plan.evolve_plan_dict(method="i1", batch_size=100) # Batch_size of 100 gives no slow-down

# for plan_dict in plan_dicts:
#     visualize_plan_dict(plan_dict, plan.meta_prompt.task)
#     break

# Manual input on main-node test cases 
main_test_cases = [
    ({"start_date": "2024-11-17", "end_date": "2024-11-23", "output_dir": "tmp"}, {"num_papers": 194, "time_taken": 480})
]


plan.spawn_test_cases_majority(main_test_cases)
plan.evolve_sub_nodes(batch_size=20)

In [ ]:
# plan.evolve_sub_nodes() # Completely stuck in the first call, debugging ... 

from methods.evolnode import EvolNode
from methods.meta_prompt import MetaPrompt, PromptMode

self = plan 
for i, node_dict in enumerate(self.plan_dict["nodes"]):
    meta_prompt = MetaPrompt(
        task=node_dict["task"],
        func_name=node_dict["name"],
        inputs=node_dict["inputs"],
        outputs=node_dict["outputs"],
        input_types=node_dict["input_types"],
        output_types=node_dict["output_types"],
        mode=PromptMode((node_dict.get("mode", "code")).lower())
    )
    test_cases = self.test_cases_dict[node_dict["name"]]
    if "fitness" in node_dict and "code" in node_dict: 
        node = EvolNode(meta_prompt, node_dict["code"], node_dict["reasoning"], get_response=self.get_response, test_cases=test_cases, fitness=node_dict["fitness"])
    else:
        node = EvolNode(meta_prompt, None, None, get_response=self.get_response, test_cases=test_cases)
        print(f"🎲 :: Evolving {node.meta_prompt.func_name} ... ({i+1}/{len(self.plan_dict['nodes'])})")
        node.evolve("i1", replace=True, max_tries=2, num_runs=2, batch_size=20) # It's funny how 30+ sec could elapse before llm inference ... (collecting prompts ?? wtf is taking so long ??)
    self.nodes.append(node)

🎲 :: Evolving extract_age ... (2/2)


Processing LLM queries: 100%|██████████| 20/20 [00:21<00:00,  1.06s/it]


 :: Total time elapsed: 21.23s, 0 errors


Processing LLM queries: 0it [00:00, ?it/s]

 :: Total time elapsed: 0.00s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.00
  🎯 Functional fitness: 0.00
  ⭐ Global fitness:     0.00
  🔄 Compiled solutions:        20
  ⏱️ Time breakdown:
     :: Query time: 2.72s
     :: Evolution time: 21.30s
     :: Evaluation time: 0.03s
     :: Total time: 24.06s


📊 Code 0: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Missing required input parameters: text
Missing required input parameters: text


📊 Code 1: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Missing required input parameters: text
Missing required input parameters: text


📊 Code 2: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Missing required input parameters: text
Missing required input parameters: text


📊 Code 3: Fitness: 0.0%
----------------------------

In [10]:
visualize_plan_dict(plan.plan_dict, plan.meta_prompt.task)

success: successfully compiled d2_output/plan_graph.d2 to d2_output/plan_graph.png in 170.887ms
